### Features explanation 

|Feature| Explanation|
|:-:|:---|
|ContactId| Unique identifier for each contact in the database.
|FirstName| The first name of the contact.
|LastName| The last name of the contact.
|FullName| The complete name of the contact, often a combination of first name and last name. 
|DateOfBirth| The contact's date of birth (Format: YYYY-MM-DD).
|Gender| The gender of the contact (e.g., Male, Female, Other).
|Email| The contact’s email address.
|Telephone| The contact’s telephone number, including country and area codes.
|PostCode| The postal code corresponding to the contact’s address.
|StreetAddress| The street address where the contact resides.
|City| The city where the contact resides.
|State| The state or province where the contact resides.
|Country| The country where the contact resides.
|Created On| The date and time when the contact was added to the system (Format: YYYY-MM-DD HH:MM).
|Headshot| A link or file reference to the contact’s headshot (image file).
|Loyalty Tier| The loyalty tier assigned to the contact (e.g., Bronze, Silver, Gold), indicating their customer status or engagement level with the company.
|Email Subscriber| Indicates whether the contact has subscribed to receive marketing emails (Yes/No).
|Income| The estimated or reported income of the contact, usually represented annually.
|Occupation| The contact's current occupation or job title.
|CustomerSatisfaction| 'high', 'medium', 'low'



### Data Load

In [3]:
import pandas as pd

def load_data(url):
    try:
        # Attempt to load the data
        base_df = pd.read_csv(url, delimiter=',', encoding='utf-8')  # Modify the delimiter or encoding if necessary
        print("Data loaded successfully. Showing the first 50 rows:")
        return base_df.head(50)

    except FileNotFoundError:
        print("Error: File not found at the specified URL.")

    except pd.errors.ParserError:
        print("Error: Could not parse the CSV file. Please check the file format and delimiter.")

    except UnicodeDecodeError:
        print("Error: Encoding issue encountered. Trying a different encoding.")
        try:
            base_df = pd.read_csv(url, delimiter=',', encoding='ISO-8859-1')
            print("Data loaded successfully using 'ISO-8859-1' encoding. Showing the first 50 rows:")
            return base_df.head(50)
        except Exception as e:
            print(f"An error occurred while loading the file with alternative encoding: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Provide the URL for testing
url = "https://filestransfer.blob.core.windows.net/ciad/Contact.txt"

# Load and display the data1
df_test = load_data(url)
if df_test is not None:
    print(df_test)

An unexpected error occurred: HTTP Error 409: Public access is not permitted on this storage account.


In [ ]:
print(base_df.columns)

### Exploratory Data Analysis

In [ ]:
# Volume of the Dataset
num_records = len(base_df)
num_features = len(base_df.columns)

print(f"The dataset has {num_records} records and {num_features} features.")

In [ ]:
print("Data types:", base_df.dtypes)

In [ ]:
print("Missing values in each column:", base_df.isnull().sum())

In [ ]:
base_df.describe()

### Missing Values
Since it was not found vas amounts of missing values we will be dropping the only feature that has them

In [ ]:
# Dropping rows with missing values in critical columns
df_cleaned = base_df.dropna(subset=[
    "DateOfBirth"
])

#### Removing Unnecessary Columns

We remove columns that do not contribute to the analysis or predictive models. By eliminating these irrelevant features, we streamline the dataset, reduce noise, and improve the efficiency of the data processing pipeline.


In [ ]:
# List of columns to drop, the ones dropped 
columns_to_drop = ['ContactId', 'FirstName', 'LastName', 'FullName', 'Gender', 'EMail', 'Telephone', 'PostCode', 'StreetAddress', 'Headshot', 'Country']

df_cleaned = df_cleaned.drop(columns=columns_to_drop)

df_cleaned.head()

### Fix Data types

In [ ]:
df_cleaned['DateOfBirth'] = pd.to_datetime(base_df['DateOfBirth'], errors='coerce')
df_cleaned['CreatedOn'] = pd.to_datetime(base_df['CreatedOn'], errors='coerce')      

In [ ]:
df_cleaned['Loyalty Tier'] = base_df['Loyalty Tier'].astype('category')
df_cleaned['CustomerSatisfaction'] = base_df['CustomerSatisfaction'].astype('category')
df_cleaned['Email Subscriber'] = base_df['Email Subscriber'].astype('category')

In [ ]:
df_cleaned.info

In [ ]:
df_cleaned.head(20)

In [ ]:
print("Data types:", df_cleaned.dtypes)

In [ ]:
# Set the count to distinct values in each categorical feature
for col_name in ['City', 'State', 'Loyalty Tier', 'Email Subscriber', 'Occupation', 'CustomerSatisfaction']:
    print(f"\nDistinct counts for {col_name}:")
    print(df_cleaned[col_name].value_counts())

In [ ]:
# Check for duplicates in the df_cleaned dataframe
duplicates = df_cleaned[df_cleaned.duplicated()]

# Display the duplicate rows, if any
if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)
else:
    print("No duplicate rows found.")

## Feature Exploration

In [ ]:
import matplotlib.pyplot as plt

loyalty_tier_counts = df_cleaned['Loyalty Tier'].value_counts()

# Plotting the distribution of Loyalty Tier
plt.figure(figsize=(8, 6))
loyalty_tier_counts.plot(kind='bar', color='skyblue')

# Adding titles and labels
plt.title('Distribution of Loyalty Tier', fontsize=14)
plt.xlabel('Loyalty Tier', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show plot
plt.xticks(rotation=45)
plt.show()

In [ ]:
customer_satisfaction_counts = df_cleaned['CustomerSatisfaction'].value_counts()

# Plotting the distribution 
plt.figure(figsize=(8, 6))
customer_satisfaction_counts.plot(kind='bar', color='skyblue')

# Adding titles and labels
plt.title('Distribution of Customer Satisfaction', fontsize=14)
plt.xlabel('Satisfaction Tier', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show plot
plt.xticks(rotation=45)
plt.show()

In [ ]:
import seaborn as sns

# Plotting the distribution of Income
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Income'], kde=True, color='green')

# Adding titles and labels
plt.title('Distribution of Income', fontsize=14)
plt.xlabel('Income', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show plot
plt.show()

In [ ]:
from datetime import datetime

# Calculate age by subtracting 'DateOfBirth' from today's date
current_date = datetime.now()
df_cleaned['Age'] = df_cleaned['DateOfBirth'].apply(lambda dob: current_date.year - dob.year - ((current_date.month, current_date.day) < (dob.month, dob.day)))

# Plotting the distribution of Age
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Age'], kde=True, color='purple')

# Adding titles and labels
plt.title('Distribution of Customer Age', fontsize=14)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show plot
plt.show()

In [ ]:
# Ensure 'CreatedOn' is in datetime format
df_cleaned['CreatedOn'] = pd.to_datetime(df_cleaned['CreatedOn'])

# Calculate tenure in days
df_cleaned['Tenure'] = (current_date - df_cleaned['CreatedOn']).dt.days

# Plotting a histogram and density plot of Customer Tenure
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Tenure'], kde=True, color='blue')

# Adding titles and labels
plt.title('Distribution of Customer Tenure', fontsize=14)
plt.xlabel('Tenure (Days)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Tenure'], kde=True, color='blue')

# Adding titles and labels
plt.title('Distribution of Customer Tenure', fontsize=14)
plt.xlabel('Tenure (Days)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Set y-axis limit to zoom in on the plot
plt.ylim(0, 5000)

# Show plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Tenure'], kde=True, color='blue')

# Adding titles and labels
plt.title('Distribution of Customer Tenure (Logarithmic Scale)', fontsize=14)
plt.xlabel('Tenure (Days)', fontsize=12)
plt.ylabel('Frequency (Log Scale)', fontsize=12)

# Set y-axis to log scale
plt.yscale('log')

# Show plot
plt.show()

In [ ]:
# Check the min and max dates in the CreatedOn feature
print("Minimum CreatedOn Date:", df_cleaned['CreatedOn'].min())
print("Maximum CreatedOn Date:", df_cleaned['CreatedOn'].max())

In [ ]:
# Check the first few rows of Tenure to verify the calculation
df_cleaned['Tenure'] = (current_date - df_cleaned['CreatedOn']).dt.days
df_cleaned[['CreatedOn', 'Tenure']].head(5000)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['CreatedOn'], kde=True, color='green')

# Adding titles and labels
plt.title('Distribution of CreatedOn Dates', fontsize=14)
plt.xlabel('CreatedOn Date', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
email_subscriber_counts = df_cleaned['Email Subscriber'].value_counts()

# Create a bar plot
email_subscriber_counts.plot(kind='bar', color='orange')

# Adding titles and labels
plt.title('Distribution of Email Subscriber', fontsize=14)
plt.xlabel('Email Subscriber Status', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show the plot
plt.xticks(rotation=0)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
occupation_counts = df_cleaned['Occupation'].value_counts()

# Create a bar plot
occupation_counts.plot(kind='bar', color='orange')

# Adding titles and labels
plt.title('Distribution of Occupations', fontsize=14)
plt.xlabel('Occupations', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show the plot
plt.xticks(rotation=0)
plt.show()

## Feature Correlations

In [ ]:
# Extract only the numerical columns
numerical_columns = df_cleaned.select_dtypes(include=['float64', 'int64']).columns

# Calculate the correlation matrix
correlation_matrix = df_cleaned[numerical_columns].corr()

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

# Adding titles and labels
plt.title('Correlation Matrix of Numerical Features', fontsize=14)
plt.show()

In [ ]:
from scipy.stats import chi2_contingency

# Extract the categorical columns
categorical_columns = df_cleaned.select_dtypes(include=['category', 'object']).columns

# Perform Chi-squared test for each pair of categorical variables
chi2_results = {}
for col1 in categorical_columns:
    for col2 in categorical_columns:
        if col1 != col2:  # We don't want to test a variable with itself
            # Create a contingency table
            contingency_table = pd.crosstab(df_cleaned[col1], df_cleaned[col2])

            # Perform the chi-squared test
            chi2, p, dof, expected = chi2_contingency(contingency_table)

            # Store the results
            chi2_results[(col1, col2)] = p

# Display significant results (p-value < 0.05)
significant_results = {key: val for key, val in chi2_results.items() if val < 0.05}
print("Significant Chi-squared tests (p-value < 0.05):")
print(significant_results)

### Fixing Imbalance

In [ ]:
import numpy as np

# We add a small constant to ensure all values are positive
df_cleaned['Income'] = df_cleaned['Income'].apply(lambda x: x + 1 if x <= 0 else x)

# Apply log transformation to the 'Income' column
df_cleaned['Log_Income'] = np.log(df_cleaned['Income'])

# Visualizing the distribution after log transformation
plt.figure(figsize=(8, 6))
sns.histplot(df_cleaned['Log_Income'], kde=True, color='blue')

# Adding titles and labels
plt.title('Distribution of Log-Transformed Income', fontsize=14)
plt.xlabel('Log(Income)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Show the plot
plt.show()

In [ ]:
# Check original Income values for log-transformed values below 10.5
lower_income_threshold = df_cleaned[df_cleaned['Log_Income'] < 10.5]['Income']

# Display those values
print(lower_income_threshold.describe())
print(lower_income_threshold.head())

In [ ]:
plt.figure(figsize=(12, 8))

# Box plot to show the Income distribution per Occupation
sns.boxplot(x='Occupation', y='Income', data=df_cleaned)

# Adding titles and labels
plt.title('Income Distribution by Occupation', fontsize=14)
plt.xlabel('Occupation', fontsize=12)
plt.ylabel('Income', fontsize=12)

# Show plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
occupation_counts = df_cleaned['Occupation'].value_counts()

# Bar plot of the number of customers per occupation
occupation_counts.plot(kind='bar', color='skyblue')

# Adding titles and labels
plt.title('Number of Customers per Occupation', fontsize=14)
plt.xlabel('Occupation', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Scatter plot to show the relationship between Income and Age
plt.scatter(df_cleaned['Age'], df_cleaned['Income'], alpha=0.5, color='green')

# Adding titles and labels
plt.title('Scatter Plot of Income vs Customer Age', fontsize=14)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Income', fontsize=12)

# Show plot
plt.show()

In [ ]:
# Create age bins (e.g., 18-25, 26-35, etc.)
age_bins = [18, 25, 35, 45, 55, 65, 75, 85]
age_labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66-75', '76-85']
df_cleaned['AgeGroup'] = pd.cut(df_cleaned['Age'], bins=age_bins, labels=age_labels)

plt.figure(figsize=(12, 8))

# Box plot to show the distribution of Income by Age Group
sns.boxplot(x='AgeGroup', y='Income', data=df_cleaned)

# Adding titles and labels
plt.title('Income Distribution by Age Group', fontsize=14)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Income', fontsize=12)

# Show plot
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Create a 2D histogram (heatmap) to visualize the density of income and age
plt.hist2d(df_cleaned['Age'], df_cleaned['Income'], bins=[30, 30], cmap='Blues')

# Adding titles and labels
plt.title('Heatmap of Income vs Customer Age', fontsize=14)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Income', fontsize=12)

# Adding color bar
plt.colorbar(label='Number of Customers')

# Show plot
plt.show()

In [ ]:
# Basic statistics for Customer Age
age_stats = df_cleaned['Age'].describe()

print("Summary Statistics for Customer Age:")
print(age_stats)

In [ ]:
plt.figure(figsize=(8, 6))

# Box plot for Customer Age
sns.boxplot(df_cleaned['Age'], color='lightblue')

# Adding titles and labels
plt.title('Box Plot of Customer Age', fontsize=14)
plt.xlabel('Age', fontsize=12)

# Show plot
plt.show()

In [ ]:
# Define age groups (bins)
age_bins = [18, 25, 35, 45, 55, 65, 75, 85]
age_labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66-75', '76-85']
df_cleaned['AgeGroup'] = pd.cut(df_cleaned['Age'], bins=age_bins, labels=age_labels)

# Count the number of customers in each age group
age_group_counts = df_cleaned['AgeGroup'].value_counts().sort_index()

# Bar plot of the number of customers in each age group
plt.figure(figsize=(10, 6))
age_group_counts.plot(kind='bar', color='lightgreen')

# Adding titles and labels
plt.title('Number of Customers by Age Group', fontsize=14)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)

# Show plot
plt.xticks(rotation=45)
plt.show()